To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


# Goal: Make Ministral solve Sudoku puzzles with Reinforcement Learning

Our goal is to make Ministral learn to solve Sudoku puzzles using reinforcement learning (GRPO).
The model will devise a strategy to fill in empty cells, and we'll reward it for correct placements
and completing valid puzzles.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Sudoku_Puzzle_by_L2G-20050714_solution_standardized_layout.svg/1280px-Sudoku_Puzzle_by_L2G-20050714_solution_standardized_layout.svg.png" height="300" />


# Installation
We'll be using [Unsloth](https://github.com/unslothai/unsloth) to do RL on Ministral. Unsloth saves 70% VRAM usage and makes reinforcement learning 2 to 6x faster.

In [1]:
# %%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
# Install transformers branch for Ministral
!pip install git+https://github.com/huggingface/transformers.git@bf3f0ae70d0e902efab4b8517fce88f6697636ce
!pip install --no-deps trl==0.22.2
!pip install z3-solver coverage

  Cloning https://github.com/huggingface/transformers.git (to revision bf3f0ae70d0e902efab4b8517fce88f6697636ce) to /tmp/pip-req-build-x1b04ywb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-x1b04ywb
  Running command git rev-parse -q --verify 'sha^bf3f0ae70d0e902efab4b8517fce88f6697636ce'
  Running command git fetch -q https://github.com/huggingface/transformers.git bf3f0ae70d0e902efab4b8517fce88f6697636ce
  Running command git checkout -q bf3f0ae70d0e902efab4b8517fce88f6697636ce
  Resolved https://github.com/huggingface/transformers.git to commit bf3f0ae70d0e902efab4b8517fce88f6697636ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Unsloth

In [2]:
from unsloth import FastVisionModel
from tracez3 import TraceParseError, TraceRLEnvironment, evaluate_trace, parse_trace_block
import torch
max_seq_length = 256 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

ministral_models = [
    "unsloth/Ministral-3-3B-Instruct-2512", # Ministral instruct models
    "unsloth/Ministral-3-8B-Instruct-2512",
    "unsloth/Ministral-3-14B-Instruct-2512",

    "unsloth/Ministral-3-3B-Reasoning-2512", # Ministral reasoning models
    "unsloth/Ministral-3-8B-Reasoning-2512",
    "unsloth/Ministral-3-14B-Reasoning-2512",

    "unsloth/Ministral-3-3B-Base-2512", # Ministral base models
    "unsloth/Ministral-3-8B-Base-2512",
    "unsloth/Ministral-3-14B-Base-2512",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    model_name = "unsloth/Ministral-3-3B-Instruct-2512",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = False, # Enable vLLM fast inference
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'max_position_embeddings'}


==((====))==  Unsloth 2025.12.4: Fast Ministral3 patching. Transformers: 5.0.0.dev0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'max_position_embeddings'}
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'max_position_embeddings'}


Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


Loading weights:   0%|          | 0/458 [00:00<?, ?it/s]

To do efficient RL, we will use [LoRA](https://arxiv.org/abs/2106.09685), which allows us to only add 1 to 5% of extra weights to the model for finetuning purposes. This allows us to save memory usage by over 60%, and yet it retains good accuracy.

In [3]:
model = FastVisionModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.vision_tower.transformer` require gradients


# RL Environment Setup

Execute strategies with time limits to prevent infinite loops.

# Data & RL task setup

Create the prompt that instructs the model to generate a Sudoku solving strategy. You can customize this to some other task for another RL task.


In [4]:
trace_env = TraceRLEnvironment()
prompt_messages = trace_env.build_messages(0)
prompt = prompt_messages[-1]["content"]

First, let's prompt the model without RL and see how it goes:

In [5]:
text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize = False,
    add_generation_prompt = True,
)

from transformers import TextStreamer
print("=" * 50)
print("BASE MODEL OUTPUT (before RL training):")
print("=" * 50)
_ = model.generate(
    **tokenizer(images=None,text=text, return_tensors = "pt").to("cuda"),
    temperature = 1.0,
    max_new_tokens = 128,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

BASE MODEL OUTPUT (before RL training):
<s><s>[SYSTEM_PROMPT]You are a program analysis assistant. Your task is to reason about control flow in a small C program and output a branch trace that either reaches a specified target line or states that the line is unreachable. Think before you answer, but NEVER include chain-of-thought in the final output.

Always respond using a single ```trace fenced block that starts with ``answer: <reachable|unreachable>`` followed by zero or more lines that encode the branch decisions along one feasible path. Each branch line must be ``<line-number> <T|F>`` where the line number refers to the numbered listing provided in the user prompt and ``T``/``F`` tell whether that branch is taken toward the target line.

If you believe the target line is unreachable, output only the answer line within the `````trace`` block. Otherwise, list the branch decisions in the order they are executed.

Example program
```
1: void foo() {
2:     int a = 0;
3:     if (a) {
4

# Reward functions

We now design a `extract_function` function which simply extracts the ` ```trace ` block.

And 1 reward functions:

1. `eval_task` which evals the prediction with z3 solver.

**Reward 1: Function Works**

Checks if the generated code is valid Python and can be executed.

In [6]:
def eval_task(completions, task, **kwargs):
    """Reward for generating valid executable Python code."""
    scores = []
    for completion, task_id in zip(completions, task):
        task = trace_env.tasks[task_id]
        score = 0
        response = completion[0]["content"]
        score = evaluate_trace(task, response)

        scores.append(score.reward)
    return scores

# Dataset Preparation

Create the training dataset.

In [7]:
from datasets import Dataset

dataset = Dataset.from_list([
    {
        "prompt": trace_env.build_messages(index),
        "task": index,
    } for index in range(3)
])

maximum_length = len(tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt.strip()}],
    add_generation_prompt=True
))

print(f"Maximum prompt length: {maximum_length}")
print("\nDataset sample:")
print(dataset[0])

Maximum prompt length: 2756

Dataset sample:
{'prompt': [{'content': 'You are a program analysis assistant. Your task is to reason about control flow in a small C program and output a branch trace that either reaches a specified target line or states that the line is unreachable. Think before you answer, but NEVER include chain-of-thought in the final output.\n\nAlways respond using a single ```trace fenced block that starts with ``answer: <reachable|unreachable>`` followed by zero or more lines that encode the branch decisions along one feasible path. Each branch line must be ``<line-number> <T|F>`` where the line number refers to the numbered listing provided in the user prompt and ``T``/``F`` tell whether that branch is taken toward the target line.\n\nIf you believe the target line is unreachable, output only the answer line within the `````trace`` block. Otherwise, list the branch decisions in the order they are executed.\n\nExample program\n```\n1: void foo() {\n2:     int a = 0;

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations! We also support GSPO, GAPO, Dr GRPO and more! Go the Unsloth [Reinforcement Learning Docs](https://docs.unsloth.ai/get-started/reinforcement-learning-rl-guide) for more options.

In [8]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

import os

# os.environ["WANDB_PROJECT"] = "ministral_3b"
# import weave; weave.init('think_test')

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    temperature = 1.0,
    learning_rate = 5e-5,
    weight_decay = 0.001,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 12,
    report_to = "none", # Can use Weights & Biases, TrackIO
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Unsloth: We now expect `per_device_train_batch_size` * `gradient_accumulation_steps` * `world_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get low reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [9]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        eval_task
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)

And let's train the model!

**NOTE** A T4 free GPU might take 5 minutes for one generation sadly since it's an old GPU - A100 or H100 will be much faster!

In [10]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3 | Num Epochs = 34 | Total steps = 100
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 67,502,080 of 3,916,592,128 (1.72% trained)
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 262144}. If this is not desired, please set these values explicitly.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / eval_task / mean,rewards / eval_task / std
1,0.000000,-0.722222,0.680414,17.000000,13.000000,19.000000,0.000000,17.000000,13.000000,19.000000,0.000000,-0.722222,0.680414
2,-0.000000,-0.583333,1.114301,17.666668,14.000000,24.000000,0.000000,17.666668,14.000000,24.000000,0.000000,-0.583333,1.114301
3,0.000114,-1.333333,0.516398,28.500000,18.000000,59.000000,0.000000,28.500000,18.000000,59.000000,0.113824,-1.333333,0.516398
4,0.000566,-1.500000,0.547723,15.166667,11.000000,23.000000,0.000000,15.166667,11.000000,23.000000,0.566257,-1.500000,0.547723
5,0.000683,-1.166667,0.408248,20.500000,12.000000,26.000000,0.000000,20.500000,12.000000,26.000000,0.682940,-1.166667,0.408248
6,0.001657,0.027778,1.240146,28.833334,13.000000,87.000000,0.000000,28.833334,13.000000,87.000000,1.656679,0.027778,1.240146
7,0.001035,0.333333,1.154701,16.000000,13.000000,23.000000,0.000000,16.000000,13.000000,23.000000,1.035381,0.333333,1.154701
8,0.001265,-0.722222,0.680414,16.000000,13.000000,22.000000,0.000000,16.000000,13.000000,22.000000,1.265105,-0.722222,0.680414
9,0.002181,-1.166667,0.408248,23.166668,18.000000,37.000000,0.000000,23.166668,18.000000,37.000000,2.181633,-1.166667,0.408248
10,0.000995,-1.000000,0.000000,26.000000,22.000000,30.000000,0.000000,26.000000,22.000000,30.000000,0.994554,-1.000000,0.000000


TrainOutput(global_step=100, training_loss=0.0006725993110861239, metrics={'train_runtime': 480.9746, 'train_samples_per_second': 1.247, 'train_steps_per_second': 0.208, 'total_flos': 0.0, 'train_loss': 0.0006725993110861239})

And now with the LoRA we just trained with GRPO - we first save the LoRA first!


In [11]:
model.save_pretrained("grpo_saved_lora")  # Local saving
tokenizer.save_pretrained("grpo_saved_lora")

['grpo_saved_lora/processor_config.json']

Verify LoRA is actually trained!


In [12]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

<a name="Inference"></a>
# Inference
Now let's try the model we just trained!

In [18]:
text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize = False,
    add_generation_prompt = True,
)

from transformers import TextStreamer

_ = model.generate(
    **tokenizer(images=None,text=text, return_tensors = "pt").to("cuda"),
    temperature = 1.0,
    max_new_tokens = 512,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<s><s>[SYSTEM_PROMPT]You are a program analysis assistant. Your task is to reason about control flow in a small C program and output a branch trace that either reaches a specified target line or states that the line is unreachable. Think before you answer, but NEVER include chain-of-thought in the final output.

Always respond using a single ```trace fenced block that starts with ``answer: <reachable|unreachable>`` followed by zero or more lines that encode the branch decisions along one feasible path. Each branch line must be ``<line-number> <T|F>`` where the line number refers to the numbered listing provided in the user prompt and ``T``/``F`` tell whether that branch is taken toward the target line.

If you believe the target line is unreachable, output only the answer line within the `````trace`` block. Otherwise, list the branch decisions in the order they are executed.

Example program
```
1: void foo() {
2:     int a = 0;
3:     if (a) {
4:         a++;
5:     }
6:     if (a == 

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [14]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [15]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
